<hr style="border:2px solid #0281c9"> </hr>

<img align="left" alt="ESO Logo" src="http://archive.eso.org/i/esologo.png">  

<div align="center">
  <h1 style="color: #0281c9; font-weight: bold;">ESO Science Archive</h1> 
  <h2 style="color: #0281c9; font-weight: bold;">Jupyter Notebooks</h2>
</div>

<hr style="border:2px solid #0281c9"> </hr>

##  **Obtain archival ESO data given RA and Dec and investigate using Aladin**

This example shows how to obtain some images towards the Galacitc Centre from the [GALACTICNUCLEUS](https://www.aanda.org/articles/aa/full_html/2019/11/aa36263-19/aa36263-19.html) project (HAWKI) and veiw these in [Aladin Sky Atlas](https://aladin.cds.unistra.fr/). 

---
**Note:** See `ESO_Introduction` and `ESO_Query` notebooks for more basic usage.

<hr style="border:2px solid #0281c9"> </hr>

In [1]:
import astroquery # import astroquery
print(f"astroquery version: {astroquery.__version__}") # check the version of astroquery

astroquery version: 0.4.11.dev10290


In [2]:
from astroquery.eso import Eso # import the ESO module from astroquery

In [3]:
eso = Eso() # create an instance of the ESO class 

In [4]:
eso.maxrec = 100    # For this example we limit the number of records to 3

# **Performing a Small Cone Search**

In [5]:
from astropy.coordinates import SkyCoord # import the SkyCoord class from the astropy.coordinates module
import astropy.units as u # import the astropy.units module

In [6]:
target = "Sgr C" # set the target 
coords = SkyCoord.from_name(target) # create a SkyCoord object from the name of the source 
radius = 2 *u.arcsec # set the radius of the search 

print(f"Coordinates of {target}: {coords}") # print the coordinates of the target

Coordinates of Sgr C: <SkyCoord (ICRS): (ra, dec) in deg
    (266.15125, -29.47027778)>


In [7]:
eso.maxrec = 3    # For this example we limit the number of records to 3

## **Reduced data cone search and download**

In [8]:
table_reduced = eso.query_surveys("195.B-0283",
                                cone_ra=coords.ra.value, 
                                cone_dec=coords.dec.value, 
                                cone_radius=radius.to("deg").value) # query the ESO archive for HAWKI data

table_reduced["target_name", "s_ra", "s_dec", "proposal_id", "instrument_name", "dp_id", "release_description"] # print the first 3 rows of the table

/opt/anaconda3/envs/tst/lib/python3.9/site-packages/pyvo/dal/query.py:341: DALOverflowWarning: Partial result set. Potential causes MAXREC, async storage space, etc.
  warn("Partial result set. Potential causes MAXREC, async storage space, etc.",


target_name,s_ra,s_dec,proposal_id,instrument_name,dp_id,release_description
,deg,deg,,,,
object,float64,float64,object,object,object,object
FAST-SGRA-19,266.16201,-29.462414,195.B-0283(D),HAWKI,ADP.2021-04-30T12:05:59.471,http://www.eso.org/rm/api/v1/public/releaseDescriptions/175
FAST-SGRA-19,266.13738,-29.449327,195.B-0283(D),HAWKI,ADP.2021-04-30T12:05:59.481,http://www.eso.org/rm/api/v1/public/releaseDescriptions/175
FAST-SGRA-19,266.16201,-29.462413,195.B-0283(D),HAWKI,ADP.2021-04-30T12:05:59.489,http://www.eso.org/rm/api/v1/public/releaseDescriptions/175


In [9]:
# Specify the parent directory; update this to match your own directory structure
output_dir = "./data/" 

# Download the data products
output_files = eso.retrieve_data(table_reduced["dp_id"][0], destination=output_dir) # download the data products
output_files

INFO: Downloading datasets ... [astroquery.eso.core]
INFO: Downloading 1 files ... [astroquery.eso.core]
INFO: Downloading file 1/1 https://dataportal.eso.org/dataPortal/file/ADP.2021-04-30T12:05:59.471 to /Users/abarnes/Library/CloudStorage/Dropbox/GitHub/astroquery_examples/data [astroquery.eso.core]
INFO: Found cached file /Users/abarnes/Library/CloudStorage/Dropbox/GitHub/astroquery_examples/data/ADP.2021-04-30T12:05:59.471.fits [astroquery.eso.core]
INFO: Done! [astroquery.eso.core]


'/Users/abarnes/Library/CloudStorage/Dropbox/GitHub/astroquery_examples/data/ADP.2021-04-30T12:05:59.471.fits'

# **Aladin Preview**

In [10]:
from ipyaladin import Aladin # Aladin Lite widget for Jupyter notebooks

We start by creating an Aladin Lite widget and setting the target position as the initial view. 

Here we show the [PanSTARRS](https://outerspace.stsci.edu/display/PANSTARRS/) survey data towards the central 0.2 degrees of the Galactic Centre (in galactic coordinates).

In [11]:
aladin = Aladin(fov=0.2, # Field of view in degrees
                survey="P/PanSTARRS/DR1/color-i-r-g", # Survey to display
                target=target, # Target to display
                coo_frame="galactic") # Coordinate frame to use
aladin

Aladin(coo_frame='galactic', survey='P/PanSTARRS/DR1/color-i-r-g')

We can now load the image we downloaded from the ESO Archive into Aladin Lite widget.

This requies the use of `fits` from `astropy.io` to open the fits image. 

**Note:** The image is displayed in the same frame as the Aladin Lite widget (above).

In [12]:
from astropy.io import fits # import the fits module from astropy.io

# hdu = fits.open(output_files)[0] # Open the FITS file downloaded from the ESO archive
aladin.add_fits(output_files, name=target, opacity=0.75) # Add the FITS file to the Aladin Lite widget

We can now save this as an image to view later.

In [13]:
aladin.save_view_as_image("./figs/aladin_GALACTICNUCLEUS.png")